In [1]:
import folium
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
from lxml import etree
import requests
from IPython.display import display, HTML
import re
import matplotlib.pyplot as plt
import time
import pickle
import os

In [3]:
API_KEY = 'RGAPI-13b30cdc-783c-47f6-85f0-d43855b5e322'
SEASON = '2015'
RANKED_QUEUE = 'RANKED_SOLO_5x5'
TEN_SECONDS_MAX = '9'
TEN_MINUTES_MAX = '490'
MIN_MATCHES = 50
matchlist = []
player_list = []

In [58]:
def getUser(summoner_name_list):
    summoner_names = []
    for summoner_name in summoner_name_list:
        url = 'https://euw.api.pvp.net/api/lol/euw/v1.4/summoner/by-name/' + summoner_name + '?api_key='+API_KEY
        info = requests.request("GET", url)
        summoner_names.append(info.json()[summoner_name]['id'])
    return summoner_names

def getStats(summoner_id):
    url = 'https://euw.api.pvp.net/api/lol/euw/v1.3/stats/by-summoner/' + summoner_id + '/ranked?season=SEASON' + SEASON + '&api_key='+API_KEY
    info = requests.request("GET", url)
    return info.json()

In [17]:
url = 'https://euw.api.pvp.net/api/lol/euw/v1.3/stats/by-summoner/' + user_id + '/ranked?season=SEASON' + SEASON + '&api_key='+API_KEY
print(url)
info = requests.request("GET", url)
info.json()


https://euw.api.pvp.net/api/lol/euw/v1.3/stats/by-summoner/52462176/ranked?season=SEASON2015&api_key=RGAPI-13b30cdc-783c-47f6-85f0-d43855b5e322


{'champions': [{'id': 5,
   'stats': {'maxChampionsKilled': 10,
    'maxNumDeaths': 9,
    'mostChampionKillsPerSession': 10,
    'mostSpellsCast': 0,
    'totalAssists': 70,
    'totalChampionKills': 50,
    'totalDamageDealt': 1140321,
    'totalDamageTaken': 287945,
    'totalDeathsPerSession': 49,
    'totalDoubleKills': 4,
    'totalFirstBlood': 0,
    'totalGoldEarned': 100122,
    'totalMagicDamageDealt': 270007,
    'totalMinionKills': 637,
    'totalPentaKills': 0,
    'totalPhysicalDamageDealt': 796076,
    'totalQuadraKills': 0,
    'totalSessionsLost': 3,
    'totalSessionsPlayed': 9,
    'totalSessionsWon': 6,
    'totalTripleKills': 0,
    'totalTurretsKilled': 8,
    'totalUnrealKills': 0}},
  {'id': 76,
   'stats': {'maxChampionsKilled': 4,
    'maxNumDeaths': 8,
    'mostChampionKillsPerSession': 4,
    'mostSpellsCast': 0,
    'totalAssists': 2,
    'totalChampionKills': 4,
    'totalDamageDealt': 123438,
    'totalDamageTaken': 24177,
    'totalDeathsPerSession': 8,


In [19]:
print (getStats(user_id).keys())

dict_keys(['modifyDate', 'champions', 'summonerId'])


In [18]:
print(getUser('kikrh18'))

{'id': 52462176, 'name': 'kikrh18', 'summonerLevel': 30, 'profileIconId': 2075, 'revisionDate': 1483747382000}


In [33]:
def getMatchList(summoner_id_list):
    matchlist = []
    for summoner_id in summoner_id_list:
        url = url = 'https://euw.api.pvp.net/api/lol/euw/v2.2/matchlist/by-summoner/' + str(summoner_id) +'?rankedQueues=' + RANKED_QUEUE + '&seasons=SEASON' + SEASON + '&api_key='+API_KEY
        summoner_matchlist = requests.request("GET", url).json()
        matchlist += [i['matchId'] for i in summoner_matchlist['matches'] if i['matchId'] not in matchlist]
    return matchlist

def getPlayersList(matchlist):
    time_ten_seconds = time.time()
    time_ten_minutes = time.time()
    players_list = []
    for match_id in matchlist:
        url = 'https://euw.api.pvp.net/api/lol/euw/v2.2/match/' + str(match_id) + '?api_key=' + API_KEY
        info = requests.request("GET", url)
        headers = info.headers
        body = info.json()
        if len(body.keys()) == 14:
            players_match=[body['participantIdentities'][i]['player']['summonerId'] for i in range(len(body['participantIdentities'])) if i not in players_list]
            players_list += players_match
            time_ten_seconds, time_ten_minutes = checkTime(headers, time_ten_seconds, time_ten_minutes)
        
    return players_list

In [117]:
def checkTime(headers, time_ten_seconds, time_ten_minutes):
    ten_seconds_limit = headers['X-Rate-Limit-Count'].split(',')[0].split(':')[0]
    ten_minutes_limit = headers['X-Rate-Limit-Count'].split(',')[1].split(':')[0]
    if ten_seconds_limit == '1':
        time_ten_seconds = time.time()
    if ten_minutes_limit == '1':
        time_ten_minutes = time.time()
    elif ten_minutes_limit == TEN_MINUTES_MAX:
        to_sleep = 600 - (time.time()-time_ten_minutes)
        if to_sleep < 0:
            to_sleep = 0
        print("Ten minutes limit! Sleeping " + str(to_sleep) + " seconds")
        time.sleep(to_sleep)
        time_ten_minutes = time.time()
    elif ten_seconds_limit == TEN_SECONDS_MAX:
        to_sleep = 10 - (time.time() - time_ten_seconds)
        if to_sleep < 0:
            to_sleep = 0
        #print("Ten seconds limit! Sleeping " + str(to_sleep) + " seconds")
        time.sleep(to_sleep)
        time_ten_seconds = time.time()
    return time_ten_seconds, time_ten_minutes

In [35]:
list_name_seeds_summoners = ["makiaveliko84",          #always Bronze
                        "limoneslocos",           #Silver -> Silver
                        "skinny1love",          #Gold015 -> Platinum16
                        "singedairlines",        #Platinum -> Platinum
                        "bestluxitaxd"]        #Diamond  -> Diamond

list_ids_seeds_summoners= getUser(list_name_seeds_summoners)

In [36]:
matchlist = getMatchList(list_ids_seeds_summoners[1:])

In [41]:
for i in range(1,len(list_ids_seeds_summoners)):
    matchlist = getMatchList([list_ids_seeds_summoners[i]])
    player_list = getPlayersList(matchlist)
    print("Loading pickle " + str(i))
    with open('./summoner_ids.pkl', 'rb') as f:
        saved = pickle.load(f)
        saved['ids'] += player_list
    print("Saving player " + str(i))
    with open('./summoner_ids.pkl', 'wb') as f:
        pickle.dump(saved, f)

Ten seconds limit! Sleeping 6.505169868469238 seconds
Ten seconds limit! Sleeping 5.473893165588379 seconds
Ten seconds limit! Sleeping 6.557063102722168 seconds
Ten seconds limit! Sleeping 0.7520039081573486 seconds
Ten seconds limit! Sleeping 6.713052988052368 seconds
Ten seconds limit! Sleeping 6.571059942245483 seconds
Loading pickle 1
Saving player 1
Ten seconds limit! Sleeping 7.702744960784912 seconds
Ten seconds limit! Sleeping 6.627630949020386 seconds
Ten seconds limit! Sleeping 6.349884986877441 seconds
Ten seconds limit! Sleeping 6.103319883346558 seconds
Ten seconds limit! Sleeping 5.899569988250732 seconds
Ten seconds limit! Sleeping 6.865742921829224 seconds
Ten seconds limit! Sleeping 4.930828094482422 seconds
Ten seconds limit! Sleeping 7.007171869277954 seconds
Ten seconds limit! Sleeping 3.8272039890289307 seconds
Ten seconds limit! Sleeping 6.9243059158325195 seconds
Ten seconds limit! Sleeping 5.558233976364136 seconds
Ten seconds limit! Sleeping 5.9224560260772705

In [68]:
if not os.path.isfile('./summoner_ids.pkl'):
    info = {'ids': player_list }
    with open('./summoner_ids.pkl', 'wb') as f:
        pickle.dump(info, f)
if not os.path.isfile('./summoner_stats.pkl'):
    info = {}
    with open('./summoner_stats.pkl', 'wb') as f:
        pickle.dump(info, f)

In [71]:
with open('./summoner_ids.pkl', 'rb') as f:
    ids_json = pickle.load(f)
    ids_list = ids_json['ids']
with open('./summoner_stats.pkl', 'rb') as f:
    stats_json = pickle.load(f)

In [127]:
time_ten_seconds = time.time()
time_ten_minutes = time.time()
counter = 18900
for summoner_id in ids_list[18900:]:
    id_ = str(summoner_id)
    if id_ in stats_json:
        counter += 1
        if counter%100 ==0:
            print(str(counter), "players done")
        continue
    if counter%100 ==0:
        with open('./summoner_stats.pkl', 'wb') as f:
            pickle.dump(stats_json, f)
        print(str(counter), "players done")

    url = 'https://euw.api.pvp.net/api/lol/euw/v1.3/stats/by-summoner/' + id_ + '/ranked?season=SEASON' + SEASON + '&api_key='+API_KEY
    info = requests.request("GET", url)
    body = info.json()
    headers = info.headers
    while (len(body)!=3):
        time_ten_seconds, time_ten_minutes = checkTime(headers, time_ten_seconds, time_ten_minutes)
        info = requests.request("GET", url)
        body = info.json()
        print(len(body))
        headers = info.headers
        if (len(body)==3):
            print("Breaking")
            break
    
    champions = body['champions']
    pref_champ = {}
    pref_champ_sessions = 0
    total_champ = {}
    discard = False
    for champ in champions:
        if champ['id'] == 0:
            if champ['stats']['totalSessionsPlayed']<MIN_MATCHES:
                discard = True
                break
            total_champs = champ 
        elif pref_champ_sessions < champ['stats']['totalSessionsPlayed']:
            pref_champ = champ
            pref_champ_sessions = champ['stats']['totalSessionsPlayed']
    if not discard:
        stats_json[id_]={'total': total_champs, 'pref_champ': pref_champ}    
    else:
        pass
        #print("Discarded")
    counter += 1
    time_ten_seconds, time_ten_minutes = checkTime(headers, time_ten_seconds, time_ten_minutes)

19000 players done
19100 players done
19200 players done
19300 players done
19400 players done
19500 players done
19600 players done
19700 players done
19800 players done
19900 players done
20000 players done


In [9]:
def getChampions():
    url = 'https://euw.api.pvp.net/api/lol/euw/v1.2/champion?api_key='+API_KEY
    info = requests.request("GET", url)
    return info.json()
def getChampionsPassive():
    url = 'https://global.api.pvp.net/api/lol/static-data/euw/v1.2/champion?dataById=true&champData=all&api_key='+API_KEY
    info = requests.request("GET", url)
    return info.json()


In [12]:
champs = getChampionsPassive()

In [18]:
champs['data']['1']

{'allytips': ['Storing a stun for use with her ultimate can turn the tide of a team fight.',
  'Striking killing blows on minions with Disintegrate enables Annie to farm extremely well early in the game.',
  "Molten Shield is a good spell to cast to work up to Annie's stun, so sometimes it's beneficial to grab at least 1 rank in it early."],
 'blurb': 'There have always been those within Noxus who did not agree with the evils perpetrated by the Noxian High Command. The High Command had just put down a coup attempt from the self-proclaimed Crown Prince Raschallion, and a crackdown on any form of ...',
 'enemytips': ["Annie's summoned bear, Tibbers, burns opposing units around himself. Try to keep your distance from him after he's been summoned.",
  'Summoner Smite can be used to help take down Tibbers.',
  "Keep an eye out for a white, swirling power around Annie. It means she's ready to unleash her stun."],
 'id': 1,
 'image': {'full': 'Annie.png',
  'group': 'champion',
  'h': 48,
  '